# IECV-g and ICEV-d passenger cars EF comparison


<b>carculator data is saved now in the "results" folder, can directly read-in w/o running carculator again</b>


- GHGenius "Exhaust Emissions" sheet: see below screenshot

- MOVES3 used in AFLEET: (https://docs.google.com/spreadsheets/d/1VslDmX5BwkZOe26DQFPWNZUFPYauMI-yu5cJFcDOl9U/edit#gid=768239429)
  - state level: 1990 to 2020 annual data
  - We use US national here: 2020 and 2025 two year data, 2020EF != as average (US State), for 2030,2035,2050 year data, using degradation factor from AFLEET to calculate EFs

- MOVES3 used in GREET (https://greet.es.anl.gov/files/update_moves3)
  - greet_year: [2000,2005,2010,2015,2020,2025,2030,2035,2050]


- carculator: incl. all 'size': ["Small", "Large", "Mini", "Lower medium", "Medium"] except SUVs & Vans from carculator, to match GREET:: car_SIDI_ICEV_gasoline

  - while all cars incl. SUV in carculator has a Maximum curb mass [kg] of 2500kg, within the range of "passenger cars" in GREET https://carculator.readthedocs.io/en/latest/modeling.html#size-classes  SUVs are seperated from cars as GREET has seperate SUV/PUT from cars, and AFLEET considers SUV as "passenger truck"
  - Table 1 Mapping of vehicle categories in GREET and MOVES3 / MOVES3 Vehicle Operation Emission Factors - GREET Model Argonne National Laboratory (.gov) https://greet.es.anl.gov/files/update_moves3


In [78]:
#!pip install carculator
!pip list -v | grep carcu

carculator                       1.8.4                 /usr/local/lib/python3.10/dist-packages pip
carculator-utils                 1.1.0                 /usr/local/lib/python3.10/dist-packages pip


In [ ]:
# need to save data / figures
from google.colab import drive
# path
drive.mount('/content/gdrive')
gdrive_path = '/content/gdrive/My Drive/02.22_FRQ_Mobilité_durable_IVEO/Project/Lit_review_resources/tools/carculator/trial_code/benchmark/'

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
from carculator import *
import matplotlib.pyplot as plt

In [ ]:
fname = f'{gdrive_path}/figs_in_disc/DC/USEPA/udds.txt'
udds = pd.read_csv(fname,header=1, sep = "\t")
#changing udds mph to kmperh
udds["kmh"] = udds["Test Time, secs Target Speed, mph"] * 1.60934
udds_kmh = np.array(udds['kmh'])

In [ ]:
cip = CarInputParameters()
cip.static()
#dcts, array = fill_xarray_from_input_parameters(cip)

In [ ]:
greet_year = [2000,2005,2010,2015,2020,2025,2030,2035,2050]

## 1. ICEV-g

In [ ]:
scope = {
    'powertrain':["ICEV-p"],
    'size': ["Small", "Large", "Mini", "Lower medium", "Medium"]
}
_, array = fill_xarray_from_input_parameters(cip, scope=scope)
array = array.interp(year=greet_year,  kwargs={'fill_value': 'extrapolate'})
cm_us = CarModel(array, cycle=udds_kmh, country="US")
cm_us.set_all()

### tailpipe emission factors GREET vs. AFLEET (MOVES3) vs. Carculator

- carculator does not seperate PM2.5 vs. PM10, only "Particulate matters direct emissions" is provided, by looking at the raw EF file (https://github.com/romainsacchi/carculator_utils/blob/master/carculator_utils/data/emission_factors/car/EF_HBEFA42_exhaust.csv), it refers to PM2.5

- carculator - CA has same EF as in US, didn't try other countries


#### change EF here then run the following code again each time: four common pollutants:  CO, VOC, NOx, PM25

In [ ]:
ef = "PM2.5"   # CO, VOC, NOx, PM2.5

In [ ]:
if ef == "CO":
  globals()['car_us_urban_%s' % ef] = cm_us.array.sel(parameter="Carbon monoxide direct emissions, urban")  #  suburban ,rural not selected, 10^3 less EF
elif ef == "VOC":
  globals()['car_us_urban_%s' % ef] = cm_us.array.sel(parameter="Non-methane hydrocarbon direct emissions, urban")
elif ef == "NOx":
  globals()['car_us_urban_%s' % ef] = cm_us.array.sel(parameter="Nitrogen oxides direct emissions, urban")
elif ef == "PM2.5":
  globals()['car_us_urban_%s' % ef] = cm_us.array.sel(parameter="Particulate matters direct emissions, urban")
#elif ef == "PM10":
#  globals()['car_us_urban_%s' % ef] = cm_us.array.sel(parameter="Particulate matters direct emissions, urban")

In [ ]:
#extracting all ts years data: car_us_urban_ts_efname = []
globals()['car_us_urban_ts_%s' % ef] = []
for y in greet_year:
  globals()['car_us_urban_ts_%s' % ef].append(np.average(globals()['car_us_urban_%s' % ef].sel(year=y).values))

print(globals()['car_us_urban_ts_%s' % ef])

[5.871462895371381e-06, 2.289991827716462e-06, 1.201659660340329e-06, 6.198458907097933e-07, 6.952650833509756e-07, 7.210880995007973e-07, 1.6104777192100586e-07, 1.5372125937008475e-07, 1.4570897377660441e-07]


In [ ]:
ef_car = pd.DataFrame({"year":greet_year,
               #"GREET": globals()['GREET_%s' % ef],
                "carculator": globals()['car_us_urban_ts_%s' % ef]
               # "AFLEET2020": globals()['AFLEET2020_%s' % ef],
               # "AFLEET2023": globals()['AFLEET2023_%s' % ef]
                                    })
#ef_car

In [ ]:
import os
fname = "ICEVgasoline_EF_Carculator" + ef + ".xlsx"
fpath = os.path.join(gdrive_path, "EF", "results/carculator/", fname)
ef_car.to_excel(fpath, index=False)

# 2. ICEV-d

In [ ]:
scope = {
    'powertrain':["ICEV-d"],
    'size': ["Small", "Large", "Mini", "Lower medium", "Medium"]
}
_, array = fill_xarray_from_input_parameters(cip, scope=scope)
array = array.interp(year=greet_year,  kwargs={'fill_value': 'extrapolate'})
cm_us_d = CarModel(array, cycle=udds_kmh, country="US")
cm_us_d.set_all()

In [ ]:
ef = "PM2.5"   # CO, VOC, NOx, PM2.5

In [ ]:
if ef == "CO":
  globals()['car_us_urban_%s' % ef] = cm_us_d.array.sel(parameter="Carbon monoxide direct emissions, urban")  #  suburban ,rural not selected, 10^3 less EF
elif ef == "VOC":
  globals()['car_us_urban_%s' % ef] = cm_us_d.array.sel(parameter="Non-methane hydrocarbon direct emissions, urban")
elif ef == "NOx":
  globals()['car_us_urban_%s' % ef] = cm_us_d.array.sel(parameter="Nitrogen oxides direct emissions, urban")
elif ef == "PM2.5":
  globals()['car_us_urban_%s' % ef] = cm_us_d.array.sel(parameter="Particulate matters direct emissions, urban")
#elif ef == "PM10":
#  globals()['car_us_urban_%s' % ef] = cm_us_d.array.sel(parameter="Particulate matters direct emissions, urban")

In [ ]:
#extracting all ts years data: car_us_urban_ts_efname = []
globals()['car_us_urban_ts_%s' % ef] = []
for y in greet_year:
  globals()['car_us_urban_ts_%s' % ef].append(np.average(globals()['car_us_urban_%s' % ef].sel(year=y).values))

print(globals()['car_us_urban_ts_%s' % ef])

[4.144887314926147e-05, 2.4923714753212012e-05, 2.3188320674525615e-05, 5.302191790327947e-07, 4.4653842846678317e-07, 4.106023520701909e-07, 3.9562368737220335e-07, 3.7732180820563063e-07, 3.6405292894231136e-07]


In [ ]:
ef_car_d = pd.DataFrame({"year":greet_year,
               #"GREET": globals()['GREET_%s' % ef],
                "carculator": globals()['car_us_urban_ts_%s' % ef]
               # "AFLEET2020": globals()['AFLEET2020_%s' % ef],
               # "AFLEET2023": globals()['AFLEET2023_%s' % ef]
                                    })

In [ ]:
fname = "ICEV_diesel_EF_Carculator" + ef + ".xlsx"
fpath = os.path.join(gdrive_path, "EF", "results/carculator/", fname)
ef_car_d.to_excel(fpath, index=False)